In [1]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

c:\Users\Shawn Kok\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
trained_model = "./llama-student-phase1"
tokenizer = AutoTokenizer.from_pretrained(trained_model)
model = AutoModelForCausalLM.from_pretrained(trained_model)
model.eval()

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=512, bi

In [3]:
# ====== Load dataset ======
def load_partition(path: str) -> Dataset:
    df = pd.read_csv(path)
    return Dataset.from_pandas(df)

dataset = load_partition("./merged_dataset.csv")

# Select the first 10 rows only
subset_df = dataset.to_pandas().head(10)

In [5]:
# Perform inference step here
def generate_output(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        output = model.generate(**inputs, max_length=200)
    return tokenizer.decode(output[0], skip_special_tokens=True)

predictions = []

for index, row in subset_df.iterrows():
    text_input = row["string"]
    
    # Generate reasoning and classification
    generated_text = generate_output(text_input)
    
    predictions.append(generated_text)

# Add predictions to dataframe and save
subset_df["generated_reasoning"] = predictions
subset_df.to_csv("first_10_inference_results.csv", index=False)

print("Inference for first 10 data points completed.")
# Save results
# dataset = dataset.add_column("generated_reasoning", predictions)
# output_csv_path = "./llama-student-phase2.csv"
# dataset.to_pandas().to_csv(output_csv_path, index=False)

# print(f"Inference completed. Results saved to {output_csv_path}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Inference for first 10 data points completed.


In [10]:
for i, prediction in enumerate(predictions):
    print(f"Index {i}: {prediction} \n")

Index 0: However, how frataxin interacts with the Fe-S cluster biosynthesis components remains unclear as direct one-to-one interactions with each component were reported (IscS [12,22], IscU/Isu1 [6,11,16] or ISD11/Isd11 [14,15]). The presence of a frataxin domain in the ISD11/Isd11 protein suggests that it may be involved in the biosynthesis of the Fe-S cluster.
The role of frataxin in the biosynthesis of the Fe-S cluster has been investigated in the context of the ISD11/Isd11 protein, which is a key component of the Fe-S cluster biosynthesis pathway in yeast [15]. The ISD11/Isd11 protein is a frataxin domain-containing protein, and it is involved in the biosynthesis of the Fe-S cluster in yeast.
The ISD11/Isd11 protein is a frataxin domain-containing protein 

Index 1: In the study by Hickey et al. (2012), spikes were sampled from the field at the point of physiological
robinson et al.: genomic regions influencing root traits in barley 11 of 13
maturity, dried, grain threshed by hand